In [1]:
import sys
sys.path.append("../../..")
import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
%load_ext autoreload
%autoreload 1
import lightning.pytorch as pl
from lightning.pytorch.callbacks import DeviceStatsMonitor
from lightning.pytorch.tuner.tuning import Tuner
import numpy as np

from ntupleReaders.clue_ntuple_reader import ClueNtupleReader

from ml.dynamic_reduction_network import DynamicReductionNetwork
%aimport ml.regression.drn.modules
%aimport ml.regression.drn.plot
%aimport ml.regression.drn.dataset_making
from ml.regression.drn.modules import *
from ml.regression.drn.dataset_making import *
%aimport ml.regression.drn.callbacks
from ml.regression.drn.callbacks import *

2023-06-16 17:25:26.480920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 17:25:26.637482: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND thi

In [13]:
reader = ClueNtupleReader("v40", "cmssw", "sim_proton_v46_patchMIP")
datamodule = DRNDataModule(reader, datasetComputationClass=RechitsTensorMaker, transformFct=None, 
                multiprocess_loader=False, keepOnGpu="cuda:2",
                batch_size=1024
                )
model = DRNModule(drn=DynamicReductionNetwork(input_dim=4,hidden_dim=25,k=10,output_dim=1,norm=torch.tensor([ 1., 1., 1., 1.])),
    loss=RatioCorrectedLoss([-0.2597882 , -0.24326517,  1.01537901]))
devices = [2]

In [14]:
trainer = pl.Trainer(accelerator="gpu", devices=devices, enable_progress_bar=True, fast_dev_run=False, logger=True,
                     #limit_train_batches=0.1,
        max_epochs=2,# profiler="simple"
             )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model=model, datamodule=datamodule,)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name        | Type                    | Params
--------------------------------------------------------
0 | drn         | DynamicReductionNetwork | 13.8 K
1 | loss_params | RatioCorrectedLoss      | 0     
--------------------------------------------------------
13.8 K    Trainable params
0         Non-trainable params
13.8 K    Total params
0.055     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [8]:
next(iter(datamodule.train_dataloader())).x

tensor([[-0.6442,  0.7502,  1.0000,  0.2822],
        [-0.9298,  0.2779,  2.0000,  0.3178],
        [-0.4322,  0.6296,  3.0000,  1.2681],
        ...,
        [-4.8588, -1.6831, 28.0000,  0.0488],
        [ 0.6504,  4.1129, 28.0000,  0.0419],
        [ 0.8166,  1.7727, 28.0000,  0.0561]], device='cuda:2')

In [ ]:
tuner = pl.tuner.tuning.Tuner()